In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>")) #makes the display bars longer and spread out 100% across the screen width

In [2]:
from vivarium import Artifact
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from db_queries import get_ids, get_outputs
import scipy.stats

# This is runs from 2020 summer--run2-TEST (India ony) 
- we identified some errors in run1 test1, so we did run1 test2 to fix them. Here we validate if those errors were fixed.  

This is the summer 2020 BEP model 001, test run updated from feb 2020 ORIGINAL version with 


1) LRI birth prevalence has been added
 
2) LBWSG affects all cause mortality instead of individual causes
 
3) updated MMN effect size on birthweight: +45.16 (32.31 to 58.02; lower 0; upper 75) from +22.4g (8.3 to 36.4g)
 
4) updated BMI effect size on birthweight: -142.93g (-232.68 to -53.18) from RR = 2 (1.5-5) which translates to about-
 
  ~ birthweight from malnourished mothers = 2743.7256193373473
 
  ~ birthweight from normal mothers = 3017.6385859634224
 
  ~ birthweight difference = 273.9129666260751

5) IFA baseline coverage (this was left out from feb 2020 original version runs)

6) additve effect sizes for supplementation groups (this was left out from  feb 2020 original version runs)

 
file location: /share/costeffectiveness/results/vivarium_gates_bep/sample_runs/bep-run1-test

Lexion: 

coutry: Indian only

BMI groups: mal and norm


In [3]:
#from SEPTEMBER 4th 2020; 09:11:49

%cd /share/costeffectiveness/results/vivarium_gates_bep/sample_runs/bep-run1-test/india/2020_09_04_09_11_49/count_data

%ls

/ihme/costeffectiveness/results/vivarium_gates_bep/sample_runs/bep-run1-test/india/2020_09_04_09_11_49/count_data
birth_weight.csv    deaths.csv           population.csv         ylds.csv
birth_weight.hdf    deaths.hdf           population.hdf         ylds.hdf
cgf_categories.csv  gestational_age.csv  state_person_time.csv  ylls.csv
cgf_categories.hdf  gestational_age.hdf  state_person_time.hdf  ylls.hdf
cgf_z_scores.csv    person_time.csv      transition_count.csv
cgf_z_scores.hdf    person_time.hdf      transition_count.hdf


In [4]:
#run 001 

output_dirs001 = ['/share/costeffectiveness/results/vivarium_gates_bep/sample_runs/bep-run1-test/india/2020_09_04_09_11_49/count_data/']

#location labels
locations = ['India']

#use in sort_values to make custum sort list. I sorted by region. 
sorter = ['India']
sorterIndex = dict(zip(sorter,range(len(sorter))))




# INDIA by scenario and treatment groups

In [13]:
#RUN 001 (baseline, mmn, bep_hd_scale_up and bep_hd_targeted)

master_bw_001 = pd.DataFrame()

for i in list(range(len(output_dirs001))):
    #birthweight
    bw = pd.read_hdf(output_dirs001[i] + 'birth_weight.hdf').reset_index().sort_values(by=['risk','measure','scenario','treatment_group','mother_status'])
    bw = bw.rename(columns={'value':'birthweight'})
    
    #population
    pop = pd.read_hdf(output_dirs001[i] + 'population.hdf').reset_index()
    pop = pop.where(pop['measure'] == 'total_population').dropna().rename(columns={'value':'total_population'}).drop(columns=['index','measure'])
    
    data = bw.merge(pop, right_on=['scenario','treatment_group','mother_status','input_draw'], left_on=['scenario','treatment_group','mother_status','input_draw'])
    data = data.groupby(['risk','measure','scenario','treatment_group','mother_status']).mean().drop(columns=['input_draw','index']).reset_index() #average over input draws
    data['pop_x_birthweight'] = data['birthweight'] * data['total_population']
    data['location'] = locations[i]
    master_bw_001 = master_bw_001.append(data)
    
master_bw_001.head()


,risk,measure,scenario,treatment_group,mother_status,birthweight,total_population,pop_x_birthweight,location
0,birth_weight,mean,baseline,bep,malnourished,0.000000,0.00,0.000000e+00,India
1,birth_weight,mean,baseline,bep,normal,0.000000,0.00,0.000000e+00,India
2,birth_weight,mean,baseline,ifa,malnourished,2801.592157,2864.72,8.025777e+06,India
3,birth_weight,mean,baseline,ifa,normal,2951.652181,14087.24,4.158063e+07,India
4,birth_weight,mean,baseline,mmn,malnourished,0.000000,0.00,0.000000e+00,India


In [6]:
master_bw_001['scenario'].unique()

array(['baseline', 'bep_hd_scale_up', 'bep_hd_targeted_scale_up',
       'mmn_scale_up'], dtype=object)

In [7]:
master_bw_001['treatment_group'].unique()

array(['bep', 'ifa', 'mmn', 'none'], dtype=object)

In [8]:
master_bw_001['measure'].unique()

array(['mean', 'proportion_below_2500g', 'sd'], dtype=object)

# ****** CHECK birthweight shift - comparison table for INDIA******


***************************************************************************************************


# *BASELINE

In [9]:
#to get the dataframe we want from run1-TEST (BASELINE):

india_bw_baseline = master_bw_001[master_bw_001['location'].isin(['India']) & master_bw_001['measure'].isin(['mean']) & master_bw_001['scenario'].isin(['baseline']) & master_bw_001['treatment_group'].isin(['ifa','none'])]
india_bw_baseline 


,risk,measure,scenario,treatment_group,mother_status,birthweight,total_population,pop_x_birthweight,location
2,birth_weight,mean,baseline,ifa,malnourished,2801.592157,2864.72,8.025777e+06,India
3,birth_weight,mean,baseline,ifa,normal,2951.652181,14087.24,4.158063e+07,India
6,birth_weight,mean,baseline,none,malnourished,2741.874627,5609.92,1.538170e+07,India
7,birth_weight,mean,baseline,none,normal,2891.942096,27438.12,7.934945e+07,India


# MMN SCALE-UP

In [10]:
#to get the dataframe we want from run1-TEST(MMN_SCALE_UP):

india_bw_mmn = master_bw_001[master_bw_001['location'].isin(['India']) & master_bw_001['measure'].isin(['mean']) & master_bw_001['scenario'].isin(['mmn_scale_up']) & master_bw_001['treatment_group'].isin(['mmn','none'])]
india_bw_mmn


,risk,measure,scenario,treatment_group,mother_status,birthweight,total_population,pop_x_birthweight,location
28,birth_weight,mean,mmn_scale_up,mmn,malnourished,2845.563286,6732.08,1.915656e+07,India
29,birth_weight,mean,mmn_scale_up,mmn,normal,2996.467911,32992.80,9.886187e+07,India
30,birth_weight,mean,mmn_scale_up,none,malnourished,2745.444400,1742.56,4.784102e+06,India
31,birth_weight,mean,mmn_scale_up,none,normal,2891.848792,8532.56,2.467487e+07,India


# BEP HD SCALE-UP UNIVERSAL

In [11]:
#to get the dataframe we want from run1-TEST (BEP_HD_SCALE_UP):

india_bw_bep_hd_universal = master_bw_001[master_bw_001['location'].isin(['India']) & master_bw_001['measure'].isin(['mean']) & master_bw_001['scenario'].isin(['bep_hd_scale_up']) & master_bw_001['treatment_group'].isin(['bep','none'])]
india_bw_bep_hd_universal

,risk,measure,scenario,treatment_group,mother_status,birthweight,total_population,pop_x_birthweight,location
8,birth_weight,mean,bep_hd_scale_up,bep,malnourished,2945.803991,6732.08,1.983139e+07,India
9,birth_weight,mean,bep_hd_scale_up,bep,normal,3071.708616,32992.80,1.013443e+08,India
14,birth_weight,mean,bep_hd_scale_up,none,malnourished,2745.444400,1742.56,4.784102e+06,India
15,birth_weight,mean,bep_hd_scale_up,none,normal,2891.848792,8532.56,2.467487e+07,India


# BEP HD TARGETED SCALE UP

In [12]:
#to get the dataframe we want from run1-TEST (BEP_HD_TARGETED_SCALE_UP):

india_bw_bep_hd_targeted= master_bw_001[master_bw_001['location'].isin(['India']) & master_bw_001['measure'].isin(['mean']) & master_bw_001['scenario'].isin(['bep_hd_targeted_scale_up']) & master_bw_001['treatment_group'].isin(['bep','mmn','none'])]
india_bw_bep_hd_targeted

,risk,measure,scenario,treatment_group,mother_status,birthweight,total_population,pop_x_birthweight,location
16,birth_weight,mean,bep_hd_targeted_scale_up,bep,malnourished,2945.803991,6732.08,1.983139e+07,India
17,birth_weight,mean,bep_hd_targeted_scale_up,bep,normal,0.000000,0.00,0.000000e+00,India
20,birth_weight,mean,bep_hd_targeted_scale_up,mmn,malnourished,0.000000,0.00,0.000000e+00,India
21,birth_weight,mean,bep_hd_targeted_scale_up,mmn,normal,2996.467911,32992.80,9.886187e+07,India
22,birth_weight,mean,bep_hd_targeted_scale_up,none,malnourished,2745.444400,1742.56,4.784102e+06,India
23,birth_weight,mean,bep_hd_targeted_scale_up,none,normal,2891.848792,8532.56,2.467487e+07,India
